# JAX Multiverse Nullification Toy / Игрушечное мультиверсное обнуление в JAX

EN: This notebook shows how to compute and minimize the multilevel foam functional with JAX.

RU: Этот ноутбук показывает, как вычислять и минимизировать многоуровневую "пену" с помощью JAX.

In [ ]:
# EN:
# Install JAX (if needed) and gra-core.
# RU:
# Установка JAX (если нужно) и gra-core.

# In Colab you might need something like:
# !pip install -q "jax[cpu]" gra-core

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

from gra_core.jax_nullification import (
    homogeneous_projector_jax,
    multilevel_phi_jax,
    gradient_step_states,
)


In [ ]:
# EN:
# Create a toy multilevel set of states psi[l] (l = 0, 1).
# RU:
# Создаём игрушечный многоуровневый набор состояний psi[l] (l = 0, 1).

key = jax.random.PRNGKey(0)
dim = 4
n_per_level = 5
levels = [0, 1]

def random_level_states(key, n, d):
    v = jax.random.normal(key, (n, d))
    v = v / (jnp.linalg.norm(v, axis=1, keepdims=True) + 1e-12)
    return v

key0, key1 = jax.random.split(key)
psi = {
    0: random_level_states(key0, n_per_level, dim),
    1: random_level_states(key1, n_per_level, dim),
}

P = homogeneous_projector_jax(dim)
projectors = {0: P, 1: P}
lambdas = {0: 1.0, 1: 1.0}


In [ ]:
# EN:
# Define a JAX loss function J = sum_l Lambda_l * Phi^{(l)}.
# RU:
# Определяем JAX-функцию потерь J = sum_l Lambda_l * Phi^{(l)}.

def loss_fn(psi_dict):
    return multilevel_phi_jax(psi_dict, projectors, levels, lambdas)

print("EN: Initial loss:", float(loss_fn(psi)))
print("RU: Начальное значение функционала:", float(loss_fn(psi)))


In [ ]:
# EN:
# Run several gradient steps on psi to reduce the foam functional.
# RU:
# Делаем несколько градиентных шагов по psi, чтобы уменьшить функционал пены.

lr = 0.1
steps = 50

loss_history = []
psi_current = {l: psi[l] for l in levels}

for t in range(steps):
    L = float(loss_fn(psi_current))
    loss_history.append(L)
    psi_current = gradient_step_states(psi_current, projectors, levels, lambdas, lr)

print("EN: Final loss:", loss_history[-1])
print("RU: Финальное значение функционала:", loss_history[-1])


In [ ]:
# EN:
# Plot loss vs iterations (log scale).
# RU:
# Рисуем функционал по итерациям (логарифмический масштаб).

plt.figure(figsize=(6, 4))
plt.plot(loss_history)
plt.yscale("log")
plt.xlabel("Iteration / Итерация")
plt.ylabel("J (log)")
plt.title("JAX GRA Multiverse Nullification / JAX-мультиверсное обнуление")
plt.grid(True)
plt.show()
